I have at my disposal the box score of every nba game since 2003. Planning to predict the winner, spread, and over under .


Just in case I mess a lot of shit up this one works, but I'm experimenting in the original. 

In [1]:
#import dependences

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense
np.random.seed(7) #why random?
from pandas import get_dummies

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('NBADATA.csv')

In [3]:
df.head()

data = df

In [4]:
data.head()

,Unnamed: 0,GAME_ID,Date,Team,Home,Away,FG,FGA,3P,3PA,FT,FTA,OR,DR,TOT,PF,ST,TO,BL,PTS,SPREAD,PLUS_MINUS,OU,TOTAL
0,4,21300001,10/29/13,Orlando Magic,0,1,36,93,9,19,6,10,13,26,17,26,10,17,6,87,12.0,-10,U,189.5
1,5,21300001,10/29/13,Indiana Pacers,1,0,34,71,7,17,22,32,10,34,17,13,4,20,18,97,-12.0,10,U,189.5
2,1,21300002,10/29/13,Chicago Bulls,0,1,35,83,7,26,18,23,11,30,23,27,11,18,4,95,5.0,-12,O,188.0
3,0,21300002,10/29/13,Miami Heat,1,0,37,72,11,20,22,29,5,35,26,21,10,18,7,107,-5.0,12,O,188.0
4,3,21300003,10/29/13,Los Angeles Clippers,0,1,41,83,8,21,13,23,10,30,27,21,11,16,4,103,-9.5,-13,O,201.5


In [5]:
data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
del data['3P'],data['3PA']
data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
del data['FG'],data['FGA']
data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
del data['Unnamed: 0'],data['GAME_ID'],data['Date'],data['Home'],data['Away'],data['PLUS_MINUS'],data['TOTAL']
del data['FT'],data['FTA']
data = get_dummies(data)


In [6]:
data.head()

,OR,DR,TOT,PF,ST,TO,BL,PTS,SPREAD,3P%,FG%,FT%,Team_Atlanta Hawks,Team_Boston Celtics,Team_Brooklyn Nets,Team_Charlotte Hornets,Team_Chicago Bulls,Team_Cleveland Cavaliers,Team_Dallas Mavericks,Team_Denver Nuggets,Team_Detroit Pistons,Team_Golden State Warriors,Team_Houston Rockets,Team_Indiana Pacers,Team_Los Angeles Clippers,Team_Los Angeles Lakers,Team_Memphis Grizzlies,Team_Miami Heat,Team_Milwaukee Bucks,Team_Minnesota Timberwolves,Team_New Orleans Pelicans,Team_New York Knicks,Team_Oklahoma City Thunder,Team_Orlando Magic,Team_Philadelphia 76ers,Team_Phoenix Suns,Team_Portland Trail Blazers,Team_Sacramento Kings,Team_San Antonio Spurs,Team_Toronto Raptors,Team_Utah Jazz,Team_Washington Wizards,OU_O,OU_P,OU_U
0,13,26,17,26,10,17,6,87,12.0,0.473684,0.387097,0.600000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,10,34,17,13,4,20,18,97,-12.0,0.411765,0.478873,0.687500,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,11,30,23,27,11,18,4,95,5.0,0.269231,0.421687,0.782609,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,5,35,26,21,10,18,7,107,-5.0,0.550000,0.513889,0.758621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,10,30,27,21,11,16,4,103,-9.5,0.380952,0.493976,0.565217,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Figure out which columns are relevant

In [7]:
dat = []
for i in range(len(data.values)):
    data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
for p in range(int(len(data.values)/2)):
    fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
    dat.append(fullboxgame)

In [ ]:
#Need to divy this big guy up into relevant sections. 
roadspread = 
#X1 = np.concatenate((dat[0,0:7],dat[0,10:21]),axis=1)  #replace that first 0 with : once ready for all. 
dat = np.array(dat)

x1 = dat[0,0:7] #road offensive rebounds to blocks
x2 = dat[0,9:42] # road 3p% to team name (hot encoded)

x3 = dat[0,45:52] #home offensive rebounds to blocks
x4  =  dat[0,54:87] #home 3p% to hot encoded team name 
roadspread = dat[0,8]
roadpts = dat[0,7]
homepts = dat[0,52]
print(dat[0,42+45:45+45])  #home offensive rebounds to blocks


#labels for over under

y = dat[:,42:45]
print(y)  #consistent with other team. 


#labels for spread
endspreadS = roadpts-homepts
openingspreadS = dat[0,8]  #spread of the home team
#labels for winner


Things to delete:

	Unnamed: 0
    GAME_ID
    Date
    Home
    Away
    Plus_Minus
    OU or total

In [ ]:
def make_network(FILENAME,sklearn=False,keras=False,normalize=True,overunder=False,spread=False,moneyline=False):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    
    """
    Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and hot encoding is done. Other stuff too probably. Such as normalization, but I 
    didn't do that!
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    
    model : object
        MLP which can predict the outcome of NBA games
        
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    #*retrospectively that doesn't make sense, could be worth changing!
    data = read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['GAME_ID'],data['Date'],data['Home'],data['Away'],data['PLUS_MINUS'],data['TOTAL']
    del data['FT'],data['FTA']
    data = get_dummies(data)

    #print(data)
    

    dat = []
    
    #reshape the dataset so now each colummn has roadstats and homestats concatenated into the same row, used for NN 
    
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    
    #convert list to array, now possible to array operations previously not possible
    dat = np.array(dat)   
    
    openingspreadS = dat[:,8] #what the predicted spread of ther game was. 
    roadpts = dat[:,7]       #column of all the points scored by road team 
    homepts = dat[:,52]
    endspreadS = roadpts-homepts  #all the final spreads of the game

    y = []
    
    if overunder:
        y = dat[:,42:45]
        #save outcomes for all, easy to show over under. 

    
    if spread:
         for j in range(len(endspreadS)):  
            openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
            endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
            if openspread + endspread <0:
                y.append(np.array([0,1,0]))  #home team covered
            elif openspread + endspread >0:
                y.append(np.array([1,0,0]))  #road covered
            else: 
                y.append(np.array([0,0,1]))  #push!

    
    if moneyline:
        for j in range(len(endspreadS)):  
            if endspreadS[j]<0:
                #means the home team had more points
                y.append(np.array([0,1]))
            else:
                y.append(np.array([1,0])) #alternatively, a road team victory. 
          

    #Now I iterated over all these, and hot encoded the labels of each to see whether or not the spread was covered
    #and by what team. 


        
    y = np.array(y)  #same explanation as above
                         
            #concatenating all the arrays, looks messy but explanation doen in another nb. 
    x1 = dat[:,0:7] #road offensive rebounds to blocks
    x2 = dat[:,9:42] # road 3p% to team name (hot encoded)

    x3 = dat[:,45:52] #home offensive rebounds to blocks
    x4  =  dat[:,54:87] #home 3p% to hot encoded team name   
    x5 = dat[:,8]
                     
    X1 = np.concatenate((x1,x2),axis=1)
    X2 = np.concatenate((x3,x4),axis=1)
    X = np.concatenate((X1,X2),axis=1)
    #since everything got out of order I have to mash it together myself. 
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    if normalize:
        
        scaler = MinMaxScaler()
        MinMaxScaler(copy=True, feature_range=(0, 1))

        scaler.fit(X)
        X = scaler.transform(X)

    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    #now to construct a model 
    if sklearn: 
        model = MLPClassifier()
        model.shuffle = True
        model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
        model.fit(X_train,y_train)
        print(model.score(X_test,y_test))
    if keras:
        print("keras NN goes here")
        model=Sequential()
        model.add(Dense(22,input_dim=np.shape(X)[1],activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(50,activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(22,activation='relu'))

        model.add(Dense(3,activation='sigmoid'))

        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=40,epochs=20,validation_split=.2)
        scores = model.evaluate(X_test,y_test)
        print(scores[1])

    
    
    return model,scaler
    

In [ ]:
nbapredictor,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,overunder=True)

In [ ]:
nbapredictor,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,spread=True)

In [ ]:
nbapredictor,scaler = make_network('NBADATA.csv',sklearn=True,keras=False,normalize=True,moneyline=True)

The next function I have is for turning the current nba team statistics (either over the entire season or some stretch) into an array of the same shape and information as the one used for the box scores above. 

In [ ]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
   # print("Here is every teams index value: ")
   # print(teams)
    return teams,data

In [ ]:
def game_maker(roadteam,hometeam,scaler):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    game = [game]
    game = scaler.transform(game)
    return game

In [ ]:
filetouse = '1718nbateamstats.csv'  #downloaded from basketball reference, and specified date 

teams, stats =make_prediction_data(filetouse)

Playoff teams + Index #
----------------------
Warriors 0
-----------
Rockets 1
-----------
Pelicans 2
-----------
Raptors 3 
-----------
Cavs 4
-----------
76ers 6
-----------
Timberwolves 7
-----------
Thunder 11
-----------
Wizards 12
-----------
Bucks 14
-----------
Trail Blazers 15
-----------
Pacers 16
-----------
Jazz 18
-----------
Celtics 19
-----------
Heat 22
-----------
Spurs 26
-----------

Now or tomorrow it would be smark to analyze all the different games, a ton have already been played, could be worth investigating what my playoff record is after round 1 for a specific series


--Some things worth noting are playoff injuries, etc. like how the sixers are never really at their full strength and same goes for warriors but whatever

In [ ]:
#returns the keras model for prediction, along with the scaling tool to normalize future data
nbapredictor,scaler = make_network('1517_boxscores.csv',sklearn=True,keras=False,normalize=True)

In [ ]:
#conference finals HO LI FUK

gswhou = game_maker(stats.values[0],stats.values[1],scaler)
hougsw = game_maker(stats.values[1],stats.values[0],scaler)

#
clebos = game_maker(stats.values[4],stats.values[19],scaler)
boscle = game_maker(stats.values[19],stats.values[4],scaler)


In [ ]:
nbapredictor.predict(clebos)


In [ ]:
nbapredictor.predict(gswhou)

In [ ]:
#west round 2
utahou = game_maker(stats.values[18],stats.values[1],scaler) #minny at houston
houuta = game_maker(stats.values[1],stats.values[18],scaler) #vice versa
nopgsw = game_maker(stats.values[2],stats.values[0],scaler)  # pelicans at gsw
gswnop = game_maker(stats.values[0],stats.values[2],scaler)


In [ ]:
nbapredictor.predict(gswnop)

In [ ]:
#western conference round 1
sasgsw = game_maker(stats.values[26],stats.values[0],scaler)
gswsas = game_maker(stats.values[0],stats.values[26],scaler)
minhou = game_maker(stats.values[7],stats.values[1],scaler)
houmin = game_maker(stats.values[1],stats.values[7],scaler)
utaokc = game_maker(stats.values[18],stats.values[11],scaler)
okcuta = game_maker(stats.values[11],stats.values[18],scaler)
noppor = game_maker(stats.values[2],stats.values[15],scaler)
pornop = game_maker(stats.values[15],stats.values[2],scaler)


In [ ]:
nbapredictor.predict(indcle)

In [ ]:
#okc uta too close in this version, gonna re run 

In [ ]:
#eastern conference round 1
miaphi = game_maker(stats.values[22],stats.values[6],scaler)
phimia = game_maker(stats.values[6],stats.values[22],scaler)
milbos = game_maker(stats.values[14],stats.values[19],scaler)
bosmil = game_maker(stats.values[19],stats.values[14],scaler)
wastor = game_maker(stats.values[12],stats.values[3],scaler)
torwas = game_maker(stats.values[3],stats.values[12],scaler)

indcle = game_maker(stats.values[16],stats.values[4],scaler)
cleind = game_maker(stats.values[4],stats.values[16],scaler)

In [ ]:
nbapredictor.predict(torwas)

In [ ]:
#Round2matchups
phibos = game_maker(stats.values[6],stats.values[19],scaler) #philly at boston
bosphi = game_maker(stats.values[19],stats.values[6],scaler) #vice versa
cletor = game_maker(stats.values[4],stats.values[3],scaler)  #cleveland at toronto
torcle = game_maker(stats.values[3],stats.values[4],scaler) #vice versa
utahou = game_maker(stats.values[18],stats.values[1],scaler) #minny at houston
houuta = game_maker(stats.values[1],stats.values[18],scaler) #vice versa
nopgsw = game_maker(stats.values[2],stats.values[0],scaler)  # pelicans at gsw
gswnop = game_maker(stats.values[0],stats.values[2],scaler)

gswsac = game_maker(stats.values[29],stats.values[0],scaler)


In [ ]:
#so who covers the various game 1s? 
nbapredictor.predict(cletor)

In [ ]:
nbapredictor.predict(nopgsw)

In [ ]:
0.8464921 + 0.45634875 + 0.10764076

In [ ]:
0.9274949 + 0.3562849 + 0.10068643

In [ ]:
minhou = game_maker(stats.values[7],stats.values[1]) #minny at houston
houmin = game_maker(stats.values[1],stats.values[7]) #vice versa

game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[13])

In [ ]:
nbapredictor.predict(np.array([game1,game2,game3]))